# Updating comparisons


This is a text cell. Start editing!




In [1]:
import org.apache.spark._
import org.apache.spark.sql.types._
import scala.reflect.runtime.universe._
import org.apache.spark.storage.StorageLevel
import org.apache.spark.rdd._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._
import group.research.aging.spark.extensions._
import group.research.aging.spark.extensions.functions._
import kernel.display.html

In [2]:
 
  val expressionsPath = "/data/results/gray-whale/Expressions"
  val byGoPath = expressionsPath + "/GO"
  val comparison = byGoPath + "/gray_whale_with_bowhead_with_minke_with_NMR_with_human_with_mouse_with_cow_full_outer_counts_extended.tsv"
  val grouped = byGoPath + "/grouped"
  val ranked = byGoPath + "/grouped/ranked"

### Adding bat



In [4]:
val batGO = spark.readTSV("/data/results/gray-whale/Expressions/GO/raw/bat_go_all.tab", true)
batGO.show(10,1000)

+----------+------------------+------------------+------------------+------------------+
|        go|       bat_liver_1|      bat_kidney_1|       bat_liver_2|      bat_kidney_2|
+----------+------------------+------------------+------------------+------------------+
|GO:0005615|104813.53711199995| 9633.763691999999|105668.70030599993| 9218.175467000005|
|GO:0005737|      45889.810949| 39124.75226900001| 43063.92459099999| 37351.04243300002|
|GO:0016021| 40258.52125200002| 49371.60719699998| 33799.39783599998| 50314.59441799999|
|GO:0005829|      39701.526823| 48503.28607600002| 35938.73908299998|48467.252029999996|
|GO:0005634|      28991.795084|38791.502254000006|26034.121111999997|36985.395011999986|
|GO:0016020|28193.223352999994|21701.492759999997|28454.926921999995|21109.941520000015|
|GO:0005576|26396.602805999995| 7023.994042999998|      25132.769251| 6699.110292999995|
|GO:0005524|26232.830242999997| 23228.16708199999|22413.050668999993| 22506.39408900001|
|GO:0004869|17508.984

In [5]:
(batGO.count, batGO.distinct.count, batGO.select("go").distinct.count)

(12981,12981,12981)

In [6]:
val goTable = spark.readTSV(comparison, true)
goTable.show(10,1000)

+----------+------------------+----------------------------------+------------------+--------------+------------+-----------------+-----------------------+-----------------------+---------------------+-----------------------------------------------+------------------+-------------------+------------------+------------------+------------------+--------------------+------------------+------------------+-----------------+------------------+--------------------+------------------+------------------+------------------+---------------------+------------------+-------------------+------------------+
|        go|         namespace|                             label|    gray_whale_avg|uniref90_count|taxons_count|transcripts_count|human_transcripts_count|mouse_transcripts_count|cow_transcripts_count|                                           type|  gray_whale_liver|bowhead_whale_liver|       minke_liver|         NMR_liver|       human_liver|mouse_totalRNA_liver|  mouse_mRNA_liver|cow_totalRNA_li

In [7]:
goTable.columns.toList.mkString("""Seq("""", """","""", """")""")

Seq("go","namespace","label","gray_whale_avg","uniref90_count","taxons_count","transcripts_count","human_transcripts_count","mouse_transcripts_count","cow_transcripts_count","type","gray_whale_liver","bowhead_whale_liver","minke_liver","NMR_liver","human

In [8]:
val goTableUpd = goTable.join(batGO, Seq("go"), "outer").select(
   "go","namespace","label","gray_whale_avg","uniref90_count","taxons_count","transcripts_count","human_transcripts_count","mouse_transcripts_count","cow_transcripts_count","type",
    "gray_whale_liver","bowhead_whale_liver","minke_liver","NMR_liver","human_liver","mouse_totalRNA_liver","mouse_mRNA_liver","cow_totalRNA_liver","cow_mRNA_liver", "bat_liver_1", "bat_liver_2",
    "gray_whale_kidney","bowhead_whale_kidney","minke_kidney","NMR_kidney","human_kidney","mouse_totalRNA_kidney","mouse_mRNA_kidney","cow_totalRNA_kidney","cow_mRNA_kidney","bat_kidney_1", "bat_kidney_2")
goTableUpd.show(10,1000)


+----------+------------------+---------------------------------------------------------+------------------+--------------+------------+-----------------+-----------------------+-----------------------+---------------------+-----------------------------------------------+------------------+-------------------+------------------+------------------+------------------+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+------------------+---------------------+------------------+-------------------+-----------------+------------------+------------------+
|        go|         namespace|                                                    label|    gray_whale_avg|uniref90_count|taxons_count|transcripts_count|human_transcripts_count|mouse_transcripts_count|cow_transcripts_count|                                           type|  gray_whale_liver|bowhead_wh

In [9]:
  val componentsUpd = goTableUpd.where("namespace = 'cellular_component'")
  val processesUpd = goTableUpd.where("namespace = 'biological_process'")
  val functionsUpd = goTableUpd.where("namespace = 'molecular_function'")

  (componentsUpd.count, processesUpd.count, functionsUpd.count, goTableUpd.count) //(2347,12638,4934,20505)

(1739,12369,4396,19098)

In [10]:
componentsUpd.writeTSV(grouped + "/updated/by_cellular_component.tab", true)
processesUpd.writeTSV(grouped + "/updated/by_biological_process.tab", true)
functionsUpd.writeTSV(grouped + "/updated/by_molecular_function.tab", true)


parts of /data/results/gray-whale/Expressions/GO/grouped/updated/by_cellular_component.tab merged!
parts of /data/results/gray-whale/Expressions/GO/grouped/updated/by_biological_process.tab merged!
parts of /data/results/gray-whale/Expressions/GO/grouped/updated/by_molecular_function.tab merged!


/data/results/gray-whale/Expressions/GO/grouped/updated/by_molecular_function.tab

In [11]:
 def rank(df: DataFrame, name: String, rankSuffix: String = "_rank") =
    df.withColumn(name + rankSuffix, org.apache.spark.sql.functions.dense_rank().over(Window.orderBy(new ColumnName(name).desc)))

  def ranks(df: DataFrame,
            names: Seq[String],
            rankSuffix: String = "_rank") = names.foldLeft(df){
    case (f, n)=> rank(f, n, rankSuffix)
  }

  val rankedCols = componentsUpd.columns.drop(11)++List("gray_whale_avg")

val rankedComponents = ranks(componentsUpd, rankedCols)
val rankedProcesses = ranks(processesUpd, rankedCols)
val rankedFunctions = ranks(functionsUpd, rankedCols)

In [12]:
rankedProcesses.show()

+----------+------------------+--------------------+------------------+--------------+------------+-----------------+-----------------------+-----------------------+---------------------+--------------------+------------------+-------------------+------------------+------------------+------------------+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+------------------+---------------------+------------------+-------------------+------------------+------------------+------------------+---------------------+------------------------+----------------+--------------+----------------+-------------------------+---------------------+-----------------------+-------------------+----------------+----------------+----------------------+-------------------------+-----------------+---------------+-----------------+--------------------------+---------------

In [13]:
rankedComponents.writeTSV(grouped + "/updated/by_cellular_component_ranked.tab", true)
rankedProcesses .writeTSV(grouped + "/updated/by_biological_process_ranked.tab", true)
rankedFunctions.writeTSV(grouped + "/updated/by_molecular_function_ranked.tab", true)


parts of /data/results/gray-whale/Expressions/GO/grouped/updated/by_cellular_component_ranked.tab merged!
parts of /data/results/gray-whale/Expressions/GO/grouped/updated/by_biological_process_ranked.tab merged!
parts of /data/results/gray-whale/Expressions/GO/grouped/updated/by_molecular_function_ranked.tab merged!


/data/results/gray-whale/Expressions/GO/grouped/updated/by_molecular_function_ranked.tab